In [1]:
import os

os.chdir("../..")

import utils

In [2]:
outputfolder = os.getcwd() + '/output/'
datafolder = os.getcwd() + '/datasets/PTB-XL/'
sampling_rate = 100
task = 'priority'
experiment_name = 'exp4'

data, raw_labels = utils.load_dataset(datafolder, sampling_rate=sampling_rate)
labels = utils.compute_label_aggregations(raw_labels, datafolder, task)
data, labels, Y, _ = utils.select_data(data, labels, task, 0, outputfolder+experiment_name+'/data/')

In [3]:
X_test = data[labels.strat_fold == 10]
y_test = Y[labels.strat_fold == 10]

X_val = data[labels.strat_fold == 9]
y_val = Y[labels.strat_fold == 9]

X_train = data[labels.strat_fold <= 8]
y_train = Y[labels.strat_fold <= 8]

n_classes = y_train.shape[1]

print(f"This experiment has {n_classes} classes")

X_train = X_train[:,:,0]
X_test = X_test[:,:,0]
X_val = X_val[:,:,0]

This experiment has 43 classes


In [4]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, GRU, TimeDistributed, Bidirectional, LeakyReLU
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,  Input, Reshape
from tensorflow.keras.layers import Conv1D, MaxPool1D, GlobalAveragePooling1D,MaxPool1D,AveragePooling1D
from tensorflow.keras import initializers, regularizers, constraints
from tensorflow.keras.layers import Layer
import numpy as np
from tensorflow.keras.layers import BatchNormalization, ReLU, MaxPooling1D, LSTM
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K

from tensorflow.keras.models import Model, Sequential


In [5]:
inputs = Input(shape=(1000,1))
x = Conv1D(filters=64, kernel_size=5, strides=1, padding='same')(inputs)
x = BatchNormalization()(x)
x = ReLU()(x)
x = MaxPooling1D(pool_size=4, strides=4)(x)

x = Conv1D(filters=128, kernel_size=5, strides=1, padding='same')(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = MaxPooling1D(pool_size=4, strides=4)(x)

x = Conv1D(filters=256, kernel_size=5, strides=1, padding='same')(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = MaxPooling1D(pool_size=4, strides=4)(x)
x = GlobalAveragePooling1D()(x)

x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(43, activation='sigmoid')(x)

model = Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1000, 1)]         0         
                                                                 
 conv1d (Conv1D)             (None, 1000, 64)          384       
                                                                 
 batch_normalization (BatchN  (None, 1000, 64)         256       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 1000, 64)          0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 250, 64)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 250, 128)          41088 

In [6]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy',threshold=0.5),
            tf.keras.metrics.Recall(name='recall'),
            tf.keras.metrics.AUC(num_thresholds=200,
                                curve="ROC",
                                summation_method='interpolation',
                                name="AUC",
                                multi_label=True,
                                label_weights=None)]
)

In [9]:
from sklearn.metrics import roc_auc_score

preds = model.predict(X_test)

print(roc_auc_score(y_test,preds))
# auc = model.evaluate(X_test, y_test)
# print(auc)

68/68 [==============================] - 1s 18ms/step
0.8287187941412683


In [7]:
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=30)

Epoch 1/30
534/534 [==============================] - 144s 230ms/step - loss: 0.1223 - accuracy: 0.9625 - recall: 0.4785 - AUC: 0.6280 - val_loss: 0.1073 - val_accuracy: 0.9659 - val_recall: 0.5391 - val_AUC: 0.6681
Epoch 2/30
534/534 [==============================] - 136s 255ms/step - loss: 0.1047 - accuracy: 0.9660 - recall: 0.5040 - AUC: 0.6925 - val_loss: 0.0990 - val_accuracy: 0.9678 - val_recall: 0.5297 - val_AUC: 0.7313
Epoch 3/30
534/534 [==============================] - 142s 266ms/step - loss: 0.1000 - accuracy: 0.9672 - recall: 0.5126 - AUC: 0.6982 - val_loss: 0.1001 - val_accuracy: 0.9676 - val_recall: 0.4628 - val_AUC: 0.7268
Epoch 4/30
534/534 [==============================] - 86s 161ms/step - loss: 0.0976 - accuracy: 0.9677 - recall: 0.5191 - AUC: 0.7215 - val_loss: 0.0938 - val_accuracy: 0.9678 - val_recall: 0.4848 - val_AUC: 0.7306
Epoch 5/30
534/534 [==============================] - 100s 188ms/step - loss: 0.0959 - accuracy: 0.9682 - recall: 0.5264 - AUC: 0.7234 - 